In [18]:
# -*- coding: utf-8 -*-
"""
@author: li
"""

import pandas as pd
import numpy as np
import networkx as nx
import csv
import os
import shutil
from sklearn.cluster import Birch
from sklearn import preprocessing
from pathlib import Path

In [26]:
SYSTEM = 'sockshop'
# SYSTEM = 'trainticket'
# Define the main directory
DATA_FOLDER = Path(f"../../../vuDevOps/data_collection/{SYSTEM}-data/")
smoothing_window = 12
TOP_K = 3

In [27]:
# Anomaly Detection
def birch_ad_with_smoothing(df, threshold):    
    anomalies = []
    df = df.filter(like='_energy')    
    # Assuming 'df' contains the filtered columns with '_energy'
    # Rename columns to remove '_energy'
    new_column_names = {col: col.replace('_energy', '') for col in df.columns}
    df = df.rename(columns=new_column_names)
    
    for svc, power in df.items():
        # No anomaly detection in db
        if svc != 'time' and 'Unnamed' not in svc and 'rabbitmq' not in svc and 'db' not in svc:
            power = power.rolling(window=smoothing_window, min_periods=1).mean()
            x = np.array(power)
            x = np.where(np.isnan(x), 0, x)
            normalized_x = preprocessing.normalize([x])

            X = normalized_x.reshape(-1,1)

            brc = Birch(branching_factor=50, n_clusters=None, threshold=threshold, compute_labels=True)
            brc.fit(X)
            brc.predict(X)

            labels = brc.labels_
            n_clusters = np.unique(labels).size
            if n_clusters > 1:
                anomalies.append(svc)
    return anomalies

In [28]:
def energy_invocations(file_path):
        
    df = pd.read_csv(file_path)

    return df

In [29]:
def attributed_graph(service_links_df):
    # build the attributed graph 
    # input: prefix of the file
    # output: attributed graph

    DG = nx.DiGraph()    
    for index, row in service_links_df.iterrows():
        source = row['source']
        destination = row['destination']
        if 'rabbitmq' not in source and 'rabbitmq' not in destination and 'db' not in destination and 'db' not in source and 'redis' not in destination and 'redis' not in source and 'ts-account-mongo' not in destination and 'ts-account-mongo' not in source:
            DG.add_edge(source, destination)
                
    return DG

In [30]:
def svc_personalization(svc, anomaly_graph, baseline_df, df):
    # svc_base, svc_metric = svc.rsplit('_', 1)

    ctn_cols = [f'{svc}_cpu', f'{svc}_memory_rss']
    max_corr = 0.01
    max_corr_metric = None  # Track the metric with the maximum correlation

    for col in ctn_cols:
        temp = abs(baseline_df[svc].corr(df[col]))     
        if temp > max_corr:
            max_corr = temp
            max_corr_metric = col.split('_')[-1]  # Extract the resource type (e.g., 'cpu' or 'memory_rss')

    # print(anomaly_graph)
    edges_weight_avg = 0.0
    num = 0
    for u, v, data in anomaly_graph.in_edges(svc, data=True):
        num = num + 1
        edges_weight_avg = edges_weight_avg + data['weight']

    edges_weight_avg  = edges_weight_avg / num

    personalization = edges_weight_avg * max_corr

    # print(f"Node: {svc}")
    # print("Baseline Data:", baseline_df[svc])
    # print("Data for Correlation:", df[col])

    # print(f"Node: {svc}, Degree: {anomaly_graph.degree(svc)}")

    return personalization, max_corr_metric

In [31]:
def anomaly_subgraph(DG, anomalies, df, alpha, metric):
    # Get the anomalous subgraph and rank the anomalous services
    # input: 
    #   DG: attributed graph
    #   anomalies: anomalous service invocations
    #   latency_df: service invocations from data collection
    #   agg_latency_dff: aggregated service invocation
    #   faults_name: prefix of csv file
    #   alpha: weight of the anomalous edge
    # output:
    #   anomalous scores 
    # Get reported anomalous nodes
    edges = []
    nodes = []
    baseline_df = pd.DataFrame()

    # for pair in anomalies:
    #     source, svc, metric = pair['source'], pair['destination'], pair['metric']
    #     source_metric = f'{source}_{metric}'
    #     svc_metric = f'{svc}_{metric}'
    #     edges.append((source_metric, svc_metric))
    #     # print(edges)
    #     # nodes.add(source)
    #     nodes.add(svc_metric)
    #     baseline_df[svc_metric] = df[svc_metric]

    for anomaly in anomalies:
        # print("Anomaly: ", anomaly)
        edge = anomaly.split('_')
        edges.append(tuple(edge))
        svc = edge[1]
        svc_metric = f'{svc}_{metric}'
        nodes.append(svc)
        baseline_df[svc] = df[svc_metric]


    nodes = set(nodes)

    personalization = {}
    metrics = {}  # Dictionary to store max_corr_metric for each node

    for node in DG.nodes():
        if node in nodes:
            personalization[node] = 0
            metrics[node] = "energy"

    # Get the subgraph of anomaly
    anomaly_graph = nx.DiGraph()
    # print(f'Nodes = {nodes}')
    # print(f"Baseline Columns: {baseline_df.columns}")
    # print(f"DataFrame Columns: {df.columns}")

    for node in nodes:
        # print(node)
        for u, v, data in DG.in_edges(node, data=True):
            edge = (u,v)
            # print(f'Incoming Edge = {edge}')
            if edge in edges:
                data = alpha
            else:
                # if baseline_df[v].nunique() == 1 or df[f'{u}_{metric}'].nunique() == 1:
                #     # print(f"No variation in {v} or {u}. Skipping correlation.")
                #     data = 0  # Default value for correlation
                # else:
                #     data = baseline_df[v].corr(df[f'{u}_{metric}'])
                # data = baseline_df[v].corr(df[f'{u}_{metric}'])
                # print(f"Available columns in df: {df.columns}")
                # print(f"Accessing column: {u}_{metric}")
                # print(f"{baseline_df[v].nunique()}, {df[f'{u}_{metric}'].nunique()}")
                data = baseline_df[v].corr(df[f'{u}_{metric}'])

            data = round(data, 3)
            anomaly_graph.add_edge(u,v, weight=data)

       # Set personalization with container resource usage
        for u, v, data in DG.out_edges(node, data=True):
            edge = (u,v)
            # print(f'Outgoing Edge = {edge}')
            if edge in edges:
                data = alpha
            else:
                # if baseline_df[u].nunique() == 1 or df[f'{v}_{metric}'].nunique() == 1:
                #     # print(f"No variation in {v}. Skipping correlation.")
                #     data = 0  # Default value for correlation
                # else:
                #     data = baseline_df[u].corr(df[f'{v}_{metric}'])
                # data = baseline_df[u].corr(df[f'{v}_{metric}'])
                # print(f"Available columns in df: {df.columns}")
                # print(f"Accessing column: {v}_{metric}")
                # verify= f"{v}_{metric}" in df.columns
                # print(verify)
                # print(f"{baseline_df[u].nunique()}, {df[f'{v}_{metric}'].nunique()}")
                data = baseline_df[u].corr(df[f'{v}_{metric}']) 
            
            data = round(data, 3)
            anomaly_graph.add_edge(u,v, weight=data)


    for node in nodes:
        max_corr, max_corr_metric = svc_personalization(node, anomaly_graph, baseline_df, df)
        personalization[node] = max_corr / anomaly_graph.degree(node)
        metrics[node] = max_corr_metric  # Store the resource type with max correlation

    anomaly_graph = anomaly_graph.reverse(copy=True)

    edges = list(anomaly_graph.edges(data=True))

    anomaly_score = nx.pagerank(anomaly_graph, alpha=0.85, personalization=personalization, max_iter=10000)

    anomaly_score = sorted(anomaly_score.items(), key=lambda x: x[1], reverse=True)

    # Combine the results: Add the resource type to the PageRank scores
    anomaly_score_with_metrics = [
        (node, score, metrics.get(node, "energy")) for node, score in anomaly_score
    ]

    return anomaly_score_with_metrics

In [32]:
def print_results(results, model, path, level):   

    # Extract the top 3 results (node, resource type)
    top_results = [f"{node}_{metric}" for node, _, metric in results[:3]]
    # Writing nodes_list to a CSV file
    csv_filename = f'{path}\{model}_{level}_results.csv'

    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Root Cause'])
        writer.writerows([[node] for node in top_results])

In [33]:
# import itertools

# normal_data = pd.read_csv("../../AD/normal_data.csv")
# anomalous_data = pd.read_csv("../../AD/anomalous_data.csv")
# normal_data.dropna(axis=1, inplace=True)
# anomalous_data.dropna(axis=1, inplace=True)
# grouped_anomalous_data = anomalous_data.groupby(['scenario', 'service', 'users', 'repetition'])

# threshold_values = {
#     "high": 0.065,
#     "low": 0.02
# }
# alpha_values = {
#     "high": 0.7,
#     "low": 0.3
# }

# for (scenario, service, users, repetition), group_df in grouped_anomalous_data:
#     for threshold_level, alpha_level in itertools.product(threshold_values.keys(), alpha_values.keys()):
        
#         ad_threshold = threshold_values[threshold_level]
#         alpha = alpha_values[alpha_level]

#         level_name = f"threshold_{threshold_level}_alpha_{alpha_level}"

#         # Filter the test data
#         temp_df = anomalous_data[
#             (anomalous_data['scenario'] == scenario) & 
#             (anomalous_data['service'] == service) & 
#             (anomalous_data['users'] == users) & 
#             (anomalous_data['repetition'] == repetition)
#         ].copy()

#         columns_to_keep = temp_df.filter(regex='(time|_energy|_cpu|_memory_rss)$').columns
#         temp_df = temp_df[columns_to_keep]

#         trial_path = f"../results/{SYSTEM}/{scenario}/{service}/{users}/{repetition}"
#         os.makedirs(trial_path, exist_ok=True)

#         anomalies = birch_ad_with_smoothing(temp_df, ad_threshold)

#         service_links_df = pd.read_csv(f'{SYSTEM}_links.csv')
        
#         source_destination_pairs = service_links_df[
#             service_links_df['source'].isin(anomalies) & 
#             service_links_df['destination'].isin(anomalies)
#         ][['source', 'destination']]
        
#         if source_destination_pairs.empty:
#             continue

#         anomalous_pairs = source_destination_pairs.apply(lambda x: '_'.join(x), axis=1).tolist()

#         DG = attributed_graph(service_links_df)
#         anomaly_score = anomaly_subgraph(DG, anomalous_pairs, temp_df, alpha, "energy")

#         print(trial_path)
#         print_results(anomaly_score, 'MicroRCA', trial_path, level_name)


../results/sockshop/scenario_A/front-end/100/repetition_1
../results/sockshop/scenario_A/front-end/100/repetition_1
../results/sockshop/scenario_A/front-end/100/repetition_10
../results/sockshop/scenario_A/front-end/100/repetition_10
../results/sockshop/scenario_A/front-end/100/repetition_11
../results/sockshop/scenario_A/front-end/100/repetition_11
../results/sockshop/scenario_A/front-end/100/repetition_12
../results/sockshop/scenario_A/front-end/100/repetition_12
../results/sockshop/scenario_A/front-end/100/repetition_13
../results/sockshop/scenario_A/front-end/100/repetition_13
../results/sockshop/scenario_A/front-end/100/repetition_14
../results/sockshop/scenario_A/front-end/100/repetition_14
../results/sockshop/scenario_A/front-end/100/repetition_15
../results/sockshop/scenario_A/front-end/100/repetition_15
../results/sockshop/scenario_A/front-end/100/repetition_16
../results/sockshop/scenario_A/front-end/100/repetition_16
../results/sockshop/scenario_A/front-end/100/repetition_17

In [34]:
normal_data = pd.read_csv("../../AD/normal_data.csv")
anomalous_data = pd.read_csv("../../AD/anomalous_data.csv")
normal_data.dropna(axis=1, inplace=True)
anomalous_data.dropna(axis=1, inplace=True)
grouped_anomalous_data = anomalous_data.groupby(['scenario', 'service', 'users', 'repetition'])

# Default parameters
# alpha = 0.55  
# ad_threshold = 0.045 

# Define hyperparameter levels
threshold_levels = {
    "high": {"ad_threshold": 0.065, "alpha": 0.7},
    "low": {"ad_threshold": 0.02, "alpha": 0.3}
}
    
fault_type = 'energy'

for (scenario, service, users, repetition), group_df in grouped_anomalous_data:
        for level_name, params in threshold_levels.items():
                # ad_threshold = params["ad_threshold"]
                ad_threshold = 0.045
                alpha = params["alpha"]
                
                # Filter the test data from the anomalous_data DataFrame for the current scenario, service, users, and repetition
                temp_df = anomalous_data[
                        (anomalous_data['scenario'] == scenario) & 
                        (anomalous_data['service'] == service) & 
                        (anomalous_data['users'] == users) & 
                        (anomalous_data['repetition'] == repetition)
                ].copy()

                # Filter columns that end with _energy, _cpu, _memory_rss
                columns_to_keep = temp_df.filter(regex='(time|_energy|_cpu|_memory_rss)$').columns

                temp_df = temp_df[columns_to_keep]

                trial_path = f"../results/{SYSTEM}/{scenario}/{service}/{users}/{repetition}"
                os.makedirs(trial_path, exist_ok=True)

                threshold = ad_threshold
                anomalies = birch_ad_with_smoothing(temp_df, threshold)
                # print("Anomalies: ", anomalies)

                service_links_df = pd.read_csv(f'{SYSTEM}_links.csv')
                anomalous_pairs = []
        
                # for anomaly in anomalies:
                #         svc = anomaly.rsplit('_', 1)[0]  # Extract the service name (e.g., 'ts-admin-order-service')
                #         metric = anomaly.rsplit('_', 1)[1]  # Extract the metric (e.g., 'energy')

                #         # Find connections involving the service
                #         matching_edges = service_links_df[
                #         (service_links_df['source'] == svc) | (service_links_df['destination'] == svc)
                #         ]

                #         for _, edge in matching_edges.iterrows():
                #                 anomalous_pairs.append({
                #                         "source": edge['source'],
                #                         "destination": edge['destination'],
                #                         "metric": metric
                #                 })

                # print(anomalous_pairs)
                # anomaly_score_by_metric = {}

                # Extract unique source-destination pairs from the CSV
                # services_anomalies = [anomaly.replace('_energy', '') for anomaly in anomalies]
                # print(service_links_df['source'].isin(services_anomalies))
                source_destination_pairs = service_links_df[service_links_df['source'].isin(anomalies) & service_links_df['destination'].isin(anomalies)][['source', 'destination']]
                if source_destination_pairs.empty:
                        continue
                anomalous_pairs = source_destination_pairs.apply(lambda x: '_'.join(x), axis=1).tolist()
                # print("Anomalous pairs: ", anomalous_pairs)

                # # Construct attributed graph
                DG = attributed_graph(service_links_df)
                anomaly_score = anomaly_subgraph(DG, anomalous_pairs, temp_df, alpha, "energy")
                print(trial_path)
                # print(anomaly_score)
                print_results(anomaly_score, 'MicroRCA', trial_path, level_name)


../results/sockshop/scenario_A/front-end/100/repetition_1
../results/sockshop/scenario_A/front-end/100/repetition_1
../results/sockshop/scenario_A/front-end/100/repetition_10
../results/sockshop/scenario_A/front-end/100/repetition_10
../results/sockshop/scenario_A/front-end/100/repetition_11
../results/sockshop/scenario_A/front-end/100/repetition_11
../results/sockshop/scenario_A/front-end/100/repetition_12
../results/sockshop/scenario_A/front-end/100/repetition_12
../results/sockshop/scenario_A/front-end/100/repetition_13
../results/sockshop/scenario_A/front-end/100/repetition_13
../results/sockshop/scenario_A/front-end/100/repetition_14
../results/sockshop/scenario_A/front-end/100/repetition_14
../results/sockshop/scenario_A/front-end/100/repetition_15
../results/sockshop/scenario_A/front-end/100/repetition_15
../results/sockshop/scenario_A/front-end/100/repetition_16
../results/sockshop/scenario_A/front-end/100/repetition_16
../results/sockshop/scenario_A/front-end/100/repetition_17